In [1]:
#! /usr/bin/python3.5



In [5]:
import os
#import GPUtil as GPU
import tensorflow as tf
#import threading
import timeit
import time
#from tensorflow import logging
from tensorflow.python.client import timeline
import multiprocessing
#import logging
from datetime import datetime 
import sys

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
gpuN=sys.argv[1]
print(gpuN)

In [6]:
a=["A","B","C","D","hh"]

In [7]:
def chunkIt(seq, num):
   
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    if num > 1:
        while last < len(seq):
            out.append(seq[int(last):int(last + avg)])
            last += avg
    else:
        return seq

    return out

In [8]:
#model(a,0)

In [9]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
    #return [x.name for x in local_device_protos]

In [10]:
import subprocess, re, os, sys #https://github.com/yaroslavvb/stuff/blob/master/notebook_util.py
def run_command(cmd):
    """Run command, return output as string."""
    
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]
    return output.decode("ascii")


In [11]:
def list_available_gpus():
    """Returns list of available GPU ids."""
    
    output = run_command("nvidia-smi -L")
    # lines of the form GPU 0: TITAN X
    gpu_regex = re.compile(r"GPU (?P<gpu_id>\d+):")
    result = []
    for line in output.strip().split("\n"):
        m = gpu_regex.match(line)
        assert m, "Couldnt parse "+line
        result.append(int(m.group("gpu_id")))
    return result

In [12]:
def gpu_memory_map(gpu_memory_file,gpu_output_file):
    """Returns map of GPU id to memory allocated on that GPU."""

    output = run_command("nvidia-smi")
    save_txt(output,gpu_output_file)
    #print("nvidia-smi",output)
    gpu_output = output[output.find("GPU Memory"):]
    #print("GPU Memory",gpu_output)
    save_txt(gpu_output,gpu_memory_file)
   
    # lines of the form
    # |    0      8734    C   python                                       11705MiB |
    memory_regex = re.compile(r"[|]\s+?(?P<gpu_id>\d+)\D+?(?P<pid>\d+).+[ ](?P<gpu_memory>\d+)MiB")
    #print("memory_regex",memory_regex)
    rows = gpu_output.split("\n")
    #print("rows",rows)
    result = {gpu_id: 0 for gpu_id in list_available_gpus()}
    #print("result",result)
    for row in gpu_output.split("\n"):
        m = memory_regex.search(row)
        if not m:
            continue
        gpu_id = int(m.group("gpu_id"))
        gpu_memory = int(m.group("gpu_memory"))
        result[gpu_id] += gpu_memory
    return result

In [13]:
def save_txt(txt,file):
    text_file = open(file, "w")
    text_file.write(txt)
    text_file.close()

# Model


In [14]:
def model_gpu(a_list,i,gpu_name):
    from datetime import datetime 
    startTime= datetime.now() 
    start = time.process_time()
    print("************Start GPU Model*************")
    
    #print("Start Time GPU",i,"Time is ",start_time,start_oclock)
    
    #import os
    #proc = os.getpid()
    #print("GPU:",gpu_name,"PID:",proc)
    with tf.device(gpu_name):
        
        start_gpu_memory=gpu_memory_map("gpu"+str(i)+"_memory_start.txt","gpu"+str(i)+"_output_start.txt")

        save_txt(str(start_gpu_memory),"gpu"+str(i)+"_memory_map.txt")

        
        
        
        #proc = os.getpid()
        #print("GPU:",gpu_name,"PID:",proc)
        from datetime import datetime
        start_time = datetime.now()
        start_oclock=time.clock()
        z=1000000000
        c=0
        
        for x in range(z):
            c= x*2    


        print("list model",a_list,i)
        
        
        #end_time = datetime.now()
        #intimeElapsed=end_time-start_time 
        #print('Duration: {}'.format(end_time - start_time))
        #print('End Devices GPU Time elpased (hh:mm:ss.ms) {}'.format(intimeElapsed))
        #model_gpu(list_file_gpu[i],i,GPU_LIST[i])
        
        start_gpu_memory=gpu_memory_map("gpu"+str(i)+"_memory_end.txt","gpu"+str(i)+"_memory_end.txt")

        save_txt(str(start_gpu_memory),"gpu"+str(i)+"_end.txt")
    end_time=time.time()
    end_oclock=time.clock()
    print("End Time GPU",i,"Time is ",end_time,end_oclock)
    timeElapsed=datetime.now()-startTime 
    elapsed = (time.process_time() - start)
    print('model_gpu Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
    print('model_gpu Time elpased (hh:mm:ss.ms) {}'.format(elapsed))
    #print("Total Time GPU",i,"Time is ",end_time-start_time,end_oclock-start_oclock)

    #return c

In [15]:
def parell_process():
    print("************Started Main Process*************")
    #proc = os.getpid()
    #print("Main Process ID:",proc)
    import multiprocessing
    config = tf.ConfigProto(device_count={"GPU": 2,"CPU":1},
                            allow_soft_placement=True,
                            inter_op_parallelism_threads=16,
                            intra_op_parallelism_threads=16,
                            use_per_session_threads=True,
                            log_device_placement=True)
    config.gpu_options.allow_growth = True


    
    start_gpu_memory=gpu_memory_map("gpu_befor_session_memory.txt","gpu_befor_session_out.txt")

    save_txt(str(start_gpu_memory),"gpu_befor_session_memory_map.txt")

    print("gpu_memory_map",start_gpu_memory)
    startTime= datetime.now() 
    start = time.process_time()
        

    with tf.Session(config=config) as sess:
        print("************Started Session Main Process*************")
        
        sess.run(tf.global_variables_initializer())
        start_gpu_memory=gpu_memory_map("gpu_after_start_session_memory.txt","gpu_after_start_session_out.txt")

        save_txt(str(start_gpu_memory),"gpu_after_start_session_memory_map.txt")
        
        GPU_LIST=get_available_gpus()
        print("GPU_LIST",GPU_LIST)
        list_file_gpu=[]
        import os

        with tf.device('/cpu'):

            processes=[]
            list_file_gpu=chunkIt(a,len(GPU_LIST))
            for i in range(len(GPU_LIST)):
                
                proc = multiprocessing.Process(target=model_gpu, args=(list_file_gpu[i],i,GPU_LIST[i]))
                processes.append(proc)
                
                
            #multiprocessing.log_to_stderr()
            #logger = multiprocessing.get_logger()
            #logger.setLevel(logging.INFO)
            # Run processes
            print("Will Start sub process")
            for p in processes:
                
                p.start()
                print("Started PID:",p.pid,"P Name:",p.name)

            # Exit the completed processes
            print("Will join sub process")
            for p in processes:
                p.join()            
                
                
            start_gpu_memory=gpu_memory_map("gpu_befor_end_session_memory.txt","gpu_befor_end_session_out.txt")

            save_txt(str(start_gpu_memory),"gpu_befor_end_session_memory_map.txt")
           
            sess.close()
    timeElapsed=datetime.now()-startTime 
    elapsed = (time.process_time() - start)
    print('Parellel Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
    print('Parellel Time elpased (hh:mm:ss.ms) {}'.format(elapsed))
    start_gpu_memory=gpu_memory_map("gpu_after_end_session_memory.txt","gpu_after_end_session_out.txt")

    save_txt(str(start_gpu_memory),"gpu_after_end_session_memory_map.txt")
    print("************Ended Main Process*************")


# Start Main Process

In [16]:
startTime= datetime.now() 
start = time.process_time()
Start=gpu_memory_map("gpu_memory_befor_start.txt","gpu_out_befor_start.txt")

save_txt(str(Start),"gpu_memory_map_befor.txt")
print("Will Start Main Process")
#multiprocessing.log_to_stderr()
#logger = multiprocessing.get_logger()
#logger.setLevel(logging.INFO)
p = multiprocessing.Process(target=parell_process)
p.start()
print("Started PID:",p.pid,"P Name:",p.name)
p.join()

tf.reset_default_graph()
print("End Main Process")
End=gpu_memory_map("gpu_memory_after_end.txt","gpu_out_after_end.txt")

save_txt(str(End),"gpu_memory_map_after.txt")
timeElapsed=datetime.now()-startTime 
elapsed = (time.process_time() - start)
print('All Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
print('All Time elpased (hh:mm:ss.ms) {}'.format(elapsed))


Will Start Main Process


[INFO/Process-1] child process calling self.run()


************Started Main Process*************
gpu_memory_map {0: 0}
************Started Session Main Process*************
GPU_LIST []


[INFO/Process-1] process shutting down
Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-15-5d2a93e30465>", line 37, in parell_process
    list_file_gpu=chunkIt(a,len(GPU_LIST))
  File "<ipython-input-7-a1e9065fe5db>", line 3, in chunkIt
    avg = len(seq) / float(num)
ZeroDivisionError: float division by zero
[INFO/Process-1] process exiting with exitcode 1


End Main Process
